## Installing libraries

In [ ]:
!pip install cohere
!pip install langchain
!pip install pypdf
!pip install tiktoken
!pip install chromadb

## Use Cohere API endpoint

In [26]:
import cohere
key = "urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg"
co = cohere.Client(key)
response = co.generate(
  model='command-nightly',
  prompt='write a description about hong kong',
  max_tokens=300,
  temperature=0.9,
  k=0,
  p=0.75,
  stop_sequences=[],
  return_likelihoods='NONE')
print('Prediction: {}'.format(response.generations[0].text))

Prediction: Hong Kong is a bustling, vibrant city located on the southern coast of China. Known for its towering skyscrapers, bustling streets, and vibrant culture, Hong Kong is a unique blend of Eastern and Western influences. The city is home to a diverse range of people, with a rich history and a dynamic future.

One of the most striking features of Hong Kong is its skyline, which is characterized by towering skyscrapers and a stunning harbor. The city is a major financial hub, with a highly developed economy and a reputation as a global center for trade and commerce. Hong Kong is also known for its efficient transportation system, which includes the iconic Star Ferry and the world-famous Peak Tram.

Beyond its urban landscape, Hong Kong also offers a wealth of natural beauty. The city is surrounded by lush, green mountains and picturesque islands, offering a range of outdoor activities such as hiking, swimming, and sailing. The city is also home to a number of beautiful parks, incl

## Create embeddings

In [27]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key=key)
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# doc_result = embeddings.embed_documents([text])
# print(doc_result)


## Document Q&A

In [28]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.llms import Cohere
import os

loader = TextLoader("./faq_mql5.txt")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

os.environ["COHERE_API_KEY"] = key

Created a chunk of size 1241, which is longer than the specified 1000
Created a chunk of size 2332, which is longer than the specified 1000
Created a chunk of size 1050, which is longer than the specified 1000
Created a chunk of size 1704, which is longer than the specified 1000


In [29]:
embeddings = CohereEmbeddings()
db = Chroma.from_documents(docs, embeddings)

## QnA testing

In [30]:
qa = VectorDBQA.from_chain_type(llm=Cohere(), chain_type="stuff", vectorstore=db)

/Users/apple/Desktop/rag-chatbot/.venv/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [31]:
query = "Who can create a trading Signal at MQL5.com? Should I pay for this?"
qa.invoke(query)

{'query': 'Who can create a trading Signal at MQL5.com? Should I pay for this?',
 'result': ' To create a signal, anyone who registers on MQL5.com can do so for free. Signals can also be created with free or paid subscriptions, both of which are free to create. '}

In [32]:
invalid_query = "where can i buy icecream?"
qa.invoke(invalid_query)

{'query': 'where can i buy icecream?',
 'result': " I don't know the answer to this unfortunately. \nI can provide you with suggestions if you let me know the location closest to you, or where you would like to buy ice cream from. "}